In [12]:
import numpy as np
from collections import defaultdict

def read_sparse_matrix_method1(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension
        d = np.zeros(n)  # Store diagonal elements
        sparse_rows = defaultdict(list)  # Store non-diagonal elements

        # Collect all elements from the file
        data = {}
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = map(float, parts)
                row, col = int(row), int(col)

                # Combine elements with the same indices
                if (row, col) in data:
                    data[(row, col)] += value
                else:
                    data[(row, col)] = value

        # Distribute elements to d and sparse_rows
        for (row, col), value in data.items():
            if row == col:
                d[row] = value  # Diagonal element
            else:
                sparse_rows[row].append((col, value))  # Off-diagonal element

    return n, d, sparse_rows


def print_matrix_info_method1(n, d, sparse_rows):
    """Print information about a sparse matrix stored using Method 1."""
    print(f"Matrix size: {n}x{n}")
    print(f"Number of diagonal elements: {np.count_nonzero(d)}")
    print(f"Number of rows with non-diagonal elements: {len(sparse_rows)}")

    total_nondiag_elements = sum(len(elements) for elements in sparse_rows.values())
    print(f"Total number of non-diagonal elements: {total_nondiag_elements}")

    # Print first few diagonal elements
    print(f"First 5 diagonal elements: {d[:5]}")

    # Print elements in first few rows
    for row in range(min(5, n)):
        if row in sparse_rows:
            print(f"Row {row}: {sparse_rows[row]}")
        else:
            print(f"Row {row}: []")

# Read matrices
n_a, d_a, sparse_rows_a = read_sparse_matrix_method1("a.txt")
print("Matrix a.txt:")
print_matrix_info_method1(n_a, d_a, sparse_rows_a)

n_b, d_b, sparse_rows_b = read_sparse_matrix_method1("b.txt")
print("\nMatrix b.txt:")
print_matrix_info_method1(n_b, d_b, sparse_rows_b)

n_aplusb, d_aplusb, sparse_rows_aplusb = read_sparse_matrix_method1("aplusb.txt")
print("\nMatrix aplusb.txt:")
print_matrix_info_method1(n_aplusb, d_aplusb, sparse_rows_aplusb)

Matrix a.txt:
Matrix size: 2025x2025
Number of diagonal elements: 2025
Number of rows with non-diagonal elements: 2025
Total number of non-diagonal elements: 13062
First 5 diagonal elements: [162.5 242.5 136.  232.5 140.5]
Row 0: [(63, 21.0), (377, 16.5), (785, 23.0), (1397, 1.5)]
Row 1: [(436, 3.5), (652, 11.0), (766, 3.5), (1390, 10.5), (1414, 15.5), (1564, 9.0), (1602, 18.0), (55, 10.0), (635, 23.0), (1045, 2.0), (1053, 14.0), (1495, 22.0)]
Row 2: [(666, 16.5), (1058, 16.0), (1493, 3.0)]
Row 3: [(208, 11.5), (308, 20.5), (1406, 21.0), (1932, 20.5), (441, 8.5), (1005, 10.0), (1727, 6.0), (1949, 12.5), (1961, 21.5)]
Row 4: [(626, 8.5), (1130, 7.0), (1264, 7.0), (2004, 10.5), (1127, 5.0), (1753, 2.0)]

Matrix b.txt:
Matrix size: 2025x2025
Number of diagonal elements: 2025
Number of rows with non-diagonal elements: 2025
Total number of non-diagonal elements: 13108
First 5 diagonal elements: [-162.5  173.   193.5  153.   142.5]
Row 0: [(154, 14.0), (456, 15.0)]
Row 1: [(182, 1.0), (636, 

In [14]:
def add_sparse_matrices_method1(n1, d1, sparse_rows1, n2, d2, sparse_rows2):
    # Check dimensions match
    if n1 != n2:
        raise ValueError(f"Matrix dimensions don't match: {n1}x{n1} and {n2}x{n2}")

    n = n1

    # Add diagonal elements
    d_sum = d1 + d2

    # Add non-diagonal elements
    sparse_rows_sum = defaultdict(list)

    # Process elements from first matrix
    for row, elements in sparse_rows1.items():
        for col, value in elements:
            sparse_rows_sum[row].append((col, value))

    # Process elements from second matrix
    for row, elements in sparse_rows2.items():
        for col, value in elements:
            # Check if this position already has a value from the first matrix
            found = False
            for i, (c, v) in enumerate(sparse_rows_sum[row]):
                if c == col:
                    # Update existing value
                    sparse_rows_sum[row][i] = (c, v + value)
                    found = True
                    break

            if not found:
                # Add new value
                sparse_rows_sum[row].append((col, value))

    return n, d_sum, sparse_rows_sum

def verify_matrix_addition(n_a, d_a, sparse_rows_a, n_b, d_b, sparse_rows_b, n_aplusb, d_aplusb, sparse_rows_aplusb, epsilon=1e-10):
    """
    Verify that matrix a + matrix b = matrix aplusb, within epsilon tolerance.
    """
    # First, compute a + b
    n_sum, d_sum, sparse_rows_sum = add_sparse_matrices_method1(n_a, d_a, sparse_rows_a, n_b, d_b, sparse_rows_b)

    # Check dimensions
    if n_sum != n_aplusb:
        print(f"❌ Dimension mismatch: sum is {n_sum}x{n_sum}, aplusb is {n_aplusb}x{n_aplusb}")
        return False

    # Check diagonal elements
    diag_diff = np.max(np.abs(d_sum - d_aplusb))
    if diag_diff > epsilon:
        print(f"❌ Diagonal elements differ by {diag_diff}, which exceeds epsilon={epsilon}")
        return False

    # Check non-diagonal elements
    for row in set(sparse_rows_sum.keys()) | set(sparse_rows_aplusb.keys()):
        # Get elements from both matrices for this row
        elements_sum = dict(sparse_rows_sum.get(row, []))
        elements_aplusb = dict(sparse_rows_aplusb.get(row, []))

        # Compare elements
        all_cols = set(elements_sum.keys()) | set(elements_aplusb.keys())
        for col in all_cols:
            val_sum = elements_sum.get(col, 0)
            val_aplusb = elements_aplusb.get(col, 0)
            diff = abs(val_sum - val_aplusb)

            if diff > epsilon:
                print(f"❌ Element at position ({row},{col}) differs by {diff}, which exceeds epsilon={epsilon}")
                return False

    print("✅ Matrix addition verified: a + b = aplusb within the specified epsilon")
    return True

# Test matrix addition and verification
n_sum, d_sum, sparse_rows_sum = add_sparse_matrices_method1(n_a, d_a, sparse_rows_a, n_b, d_b, sparse_rows_b)
print("\nComputed sum (a + b):")
print_matrix_info_method1(n_sum, d_sum, sparse_rows_sum)

# Verify addition
print("\nVerifying that a + b = aplusb:")
verify_matrix_addition(n_a, d_a, sparse_rows_a, n_b, d_b, sparse_rows_b, n_aplusb, d_aplusb, sparse_rows_aplusb)


Computed sum (a + b):
Matrix size: 2025x2025
Number of diagonal elements: 2024
Number of rows with non-diagonal elements: 2025
Total number of non-diagonal elements: 26121
First 5 diagonal elements: [  0.  415.5 329.5 385.5 283. ]
Row 0: [(63, 21.0), (377, 16.5), (785, 23.0), (1397, 1.5), (154, 14.0), (456, 15.0)]
Row 1: [(436, 3.5), (652, 11.0), (766, 3.5), (1390, 10.5), (1414, 15.5), (1564, 9.0), (1602, 18.0), (55, 10.0), (635, 23.0), (1045, 2.0), (1053, 14.0), (1495, 22.0), (182, 1.0), (636, 22.5), (23, 11.0), (463, 23.0), (1965, 15.0)]
Row 2: [(666, 16.5), (1058, 16.0), (1493, 3.0), (1298, 4.0), (81, 8.5), (569, 15.5), (601, 14.0), (939, 5.5), (1041, 21.5), (1133, 1.0), (1305, 8.5), (1975, 14.5)]
Row 3: [(208, 11.5), (308, 20.5), (1406, 21.0), (1932, 20.5), (441, 8.5), (1005, 10.0), (1727, 6.0), (1949, 12.5), (1961, 21.5), (1204, 20.0), (1388, 12.0), (695, 20.5)]
Row 4: [(626, 8.5), (1130, 7.0), (1264, 7.0), (2004, 10.5), (1127, 5.0), (1753, 2.0), (842, 3.5), (1238, 6.5), (363, 13

True

In [21]:
def print_matrix_rows_method1(n, d, sparse_rows, start_row=0, num_rows=5):
    """Print specific rows of a matrix stored in Method 1 format."""
    print(f"Showing rows {start_row} to {min(start_row+num_rows-1, n-1)}:")

    for row in range(start_row, min(start_row+num_rows, n)):
        # Start with diagonal element
        elements = []
        if abs(d[row]) > 1e-10:  # Only include non-zero diagonal
            elements.append((row, d[row]))

        # Add non-diagonal elements
        if row in sparse_rows:
            for col, val in sparse_rows[row]:
                elements.append((col, val))

        # Sort by column
        elements.sort()

        # Print formatted row
        print(f"Row {row}: {elements}")

# Compute the sum using Method 1
n_sum, d_sum, sparse_rows_sum = add_sparse_matrices_method1(n_a, d_a, sparse_rows_a, n_b, d_b, sparse_rows_b)

# Print the first few rows of the computed sum
print("First rows from computed sum (a + b):")
print_matrix_rows_method1(n_sum, d_sum, sparse_rows_sum, 0, 5)

# Print the first few rows from aplusb.txt
print("\nFirst rows from aplusb.txt:")
print_matrix_rows_method1(n_aplusb, d_aplusb, sparse_rows_aplusb, 0, 5)

First rows from computed sum (a + b):
Showing rows 0 to 4:
Row 0: [(63, 21.0), (154, 14.0), (377, 16.5), (456, 15.0), (785, 23.0), (1397, 1.5)]
Row 1: [(1, np.float64(415.5)), (23, 11.0), (55, 10.0), (182, 1.0), (436, 3.5), (463, 23.0), (635, 23.0), (636, 22.5), (652, 11.0), (766, 3.5), (1045, 2.0), (1053, 14.0), (1390, 10.5), (1414, 15.5), (1495, 22.0), (1564, 9.0), (1602, 18.0), (1965, 15.0)]
Row 2: [(2, np.float64(329.5)), (81, 8.5), (569, 15.5), (601, 14.0), (666, 16.5), (939, 5.5), (1041, 21.5), (1058, 16.0), (1133, 1.0), (1298, 4.0), (1305, 8.5), (1493, 3.0), (1975, 14.5)]
Row 3: [(3, np.float64(385.5)), (208, 11.5), (308, 20.5), (441, 8.5), (695, 20.5), (1005, 10.0), (1204, 20.0), (1388, 12.0), (1406, 21.0), (1727, 6.0), (1932, 20.5), (1949, 12.5), (1961, 21.5)]
Row 4: [(4, np.float64(283.0)), (363, 13.0), (626, 8.5), (842, 3.5), (1127, 5.0), (1130, 7.0), (1238, 6.5), (1264, 7.0), (1753, 2.0), (1937, 19.0), (2004, 10.5)]

First rows from aplusb.txt:
Showing rows 0 to 4:
Row 0: [